In [ ]:
#1)

#This script updates the AWA rankings and records sheets with each competition

#It is important to note that this notebook is meant to run in google colab and interact with a google drive
#If you want to run locally, you will need to make some modifications

#Last modified on September 14th, 2025 by M. H. Kent

In [ ]:
#2)

#Import needed libraries to run the script

#Install needed packages that are not already built in
!pip install XlsxWriter
!pip install rapidfuzz
!pip install gspread_formatting

#Import needed packages
import pandas as pd
import numpy as np
import glob
import re
import os
import math
from datetime import datetime
from google.colab import drive
from google.colab import files
from google.colab import auth
import openpyxl as opxl
from openpyxl.styles import Font, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl.cell.cell import MergedCell
import xlsxwriter
from rapidfuzz.distance import Levenshtein
from rapidfuzz import process, fuzz
from datetime import datetime
import time
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
import google.auth
from gspread_dataframe import get_as_dataframe


In [ ]:
#3)

#Conect to the drive
#Run through the prompts that pop up. Accept what they ask and you will have full conection

#Authenticate to the user
#auth.authenticate_user() #Unhash if you have never mounted before or you have issues

#Mount to the drive
drive.mount('/content/drive')

In [ ]:
#4)

#Setting all of the nessisary paths and specifications to run the updates

#Determine what to update online
#Set as "True" if we want to update on a website or the core docs in a google drive, "False" otherwise
onl_upd_recs = False #Reccord sheet
onl_upd_rank = False #Ranking sheet

#Set paths to each sheet we need to preform updates on or upload from
#Note that the input and output path will be the same for the updates sheets
#Make sure that the file types are correct, excell vs sheets is finiky
#Note that you will need to change these paths to your own
#GO TO CELL 6 WHEN YOU KNOW IF YOU ARE USEING .XLSX FILES OR GOOGLE SHEETS

#Update sheet
res_to_add_path = "/content/drive/your sheet path here"

#Records sheet
rec_sheet_path = "/content/drive/your sheet path here"

#Rankings sheet
#rank_sheet_path = "Your path here"
rank_sheet_ID = "your google sheet id here" #This is because a google sheet was used here in the original awa code

#Classification sheet
class_sheet_path = "/content/drive/your sheet path here"

#Sinclair coefficent sheet
sinc_sheet_path = "/content/drive/you sheet path here"

#Q-Points masters age factors sheets
Q_agef_sheet = "/content/drive/your sheet path here"


In [ ]:
#5)

#Define all needed functions


#Define needed functions for Q-points and Sinclair

#q_points formulas found here https://osf.io/e6srt
def q_points_male(total, bodyweight):
  """
  Computes the Q points score for men

  total: The total hit in kg
  bodyweight: The body weight
  """
  bw_ratio = bodyweight / 100
  denominator = 416.70 - 47.87 * bw_ratio**(-2) + 18.93 * bw_ratio**2
  return (total * 463.26) / denominator

def q_points_female(total, bodyweight):
  """
  Computes the Q points score for woman

  total: The total hit in kg
  bodyweight: The body weight
  """
  bw_ratio = bodyweight / 100
  denominator = 266.50 - 19.44 * bw_ratio**(-2) + 18.61 * bw_ratio**2
  return (total * 306.54) / denominator

def qbw_male(bodyweight):
  """
  Computes the Q point bodyweight factor for men

  bodyweight: The body weight
  """
  bw_ratio = bodyweight / 100
  denominator = 416.70 - 47.87 * bw_ratio**(-2) + 18.93 * bw_ratio**2
  return 463.26 / denominator

def qbw_female(bodyweight):
  """
  Computes the Q points bodyweight factor for woman

  bodyweight: The body weight
  """
  bw_ratio = bodyweight / 100
  denominator = 266.50 - 19.44 * bw_ratio**(-2) + 18.61 * bw_ratio**2
  return 306.54 / denominator

def sinclair_coef(x, A, b):
  """
  Defines the sinclair coefficent formula

  x: The athletes bodyweight
  A: The sinclair coefficent A value
  b: The sinclair coefficent b value
  """
  div_const = float(float(x)/float(b))
  big_X = math.log10(div_const)
  if x <= b:
      return 10**(A*(big_X**2))
  if x > b:
      return 1

def sinclair_masters_mod(Ag, Age):
  """
  Defines the sinclair age coefficent modeification for masters athletes

  Age: The athletes age
  Ag: The sinclair age coefficent
  """
  return 1 + ((Ag/100)*(Age - 30))

def sinclair_total(total, x, A, b):
  """
  Calculates the sinclair total from the coefficent

  total: The athletes total hit
  x: The athletes bodyweight
  A: The sinclair coefficent A value
  b: The sinclair coefficent b value
  """
  return total * sinclair_coef(x, A, b)

def masters_sinclair_total(total, x, A, b, Ag, Age):
  """
  Calculates the sinclair total from the coefficent

  total: The athletes total hit
  x: The athletes bodyweight
  A: The sinclair coefficent A value
  b: The sinclair coefficent b value
  """
  return total * sinclair_coef(x, A, b) * sinclair_masters_mod(Ag, Age)

def list_rounder(num_list):
  """
  Rounds each float in a list to the nearest whole integer

  num_list: A list of floats
  """
  return [round(num) for num in num_list]


#Define needed functions for file uploads and basic cleaning

def sheet_dict_creater(pathID, subsheets, type, auth):
  """
  Uploads the specified subsheets from a path or an ID depending on the file type
  This function is currently equipt to handle .xlsx and .gsheet files

  pathID: The file id of the .gsheet files or the full path of the .xlxs file
  subsheet: A list of subsheet names as a string
  type: Spefifies if we are working with a .gsheet or .xlsx file. The possible inputs are "gsheet" and "xlsx"
  auth: Specifies if we need to remount to the drive or authenticate the user for the google sheet upload. "True" Means that we do
  """
  if type == "gsheet":
    if auth == True:
      from google.colab import auth
      import google.auth
      auth.authenticate_user()
      creds, _ = google.auth.default()
      gc = gspread.authorize(creds)
    file_dict = {}
    for sheet in subsheets:
      sheet_file = gc.open_by_key(pathID).worksheet(sheet)
      file_dict[sheet] = get_as_dataframe(sheet_file, evaluate_formulas=True)
    return file_dict
  elif type == "xlsx":
    if auth == True:
      drive.mount('/content/drive')
    file_dict = {}
    for sheet in subsheets:
      #print(sheet)
      file_dict[sheet] = pd.read_excel(pathID, sheet_name = sheet)
    return file_dict

def rank_sheet_mod1(rank_dict):
  """
  Fixes the ranking sheet so we preform the nessisary updates

  rank_dict: The raw ranking sheet dictionary
  """
  clean_sheet_dict = {}
  for keys in list(rank_dict.keys()):
    sheet = rank_dict[keys]
    sheet = sheet.reset_index(drop = True)
    sheet.columns = sheet.iloc[0]
    sheet = sheet[1:]
    clean_sheet_dict[keys] = sheet
  return clean_sheet_dict

def rec_sheet_mod1(rec_dict):
  """
  Fixes the ranking sheet so we preform the nessisary updates

  rec_dict: The raw ranking sheet dictionary
  """
  clean_sheet_dict = {}
  for keys in list(rec_dict.keys()):
    sheet = rec_dict[keys]
    sheet = sheet.reset_index(drop = True)
    sheet.columns = sheet.iloc[0]
    sheet = sheet[1:]
    clean_sheet_dict[keys] = sheet
  return clean_sheet_dict

def upd_sheet_colbuild(cnames, rwone):
  """
  Builds the columns of the update sheet from the first row and the original column names
  The goal is for us to be able to identify the columns that we need for our updates from this
  We assume that both the first row and the cnames list are the same

  cnames: The initial colnames of the sheet
  rwone: The first row of the update dataframe
  """
  new_cnames = []
  for i in range(len(cnames)):
    cname = cnames[i]
    rval = rwone[i]
    if 'Unnamed:' in cname:
      new_cnames.append(str(rval)) #Make sure it is a string
    else:
      new_cnames.append(cname)
  return new_cnames

def sn_cj_colname_mod(colnmlst):
  """
  Modifies any list of column names that has only integer values for snatch and clean and jerk column names if they exist

  colnmlst: The list of column names that we want to preform the mod for
  """
  new_cnames = []
  ider = 0
  for i in range(len(colnmlst)):
    cname = str(colnmlst[i])
    if 'Snatch' in cname:
      new_cnames.append('Snatch_1')
      ider = 1
    elif 'Clean&Jerk' in cname:
      new_cnames.append('Clean&Jerk_1')
      ider = 2
    elif ider == 1 and cname in ['2', '3', 'max']:
      new_name = 'Snatch_' + cname
      new_cnames.append(new_name)
    elif ider == 2 and cname in ['2', '3', 'max']:
      new_name = 'Clean&Jerk_' + cname
      new_cnames.append(new_name)
    else:
      new_cnames.append(cname)
      ider = 0
  return new_cnames

def upd_sheet_mod1(upd_sheet):
  """
  Modifies the update sheet so we can do the updates to the rankings and records

  upd_sheet: The update sheet as a pandas dataframe
  """
  sheet = upd_sheet.reset_index(drop = True)
  columns = list(sheet.columns)
  row_1 = list(sheet.iloc[0])
  cnames_1 = upd_sheet_colbuild(columns, row_1)
  cnames_2 = sn_cj_colname_mod(cnames_1)
  sheet.columns = cnames_2
  sheet = sheet[1:]
  return sheet

def classification_setup(file_path, sheet_nm):
  """
  Uploads the classification sheet and gets it into the correct format to update

  file_path: The path to the classification sheet
  sheet_nm: The sheet name of the classification sheet within the larger file
  """
  mens_class_base = pd.read_excel(file_path, sheet_name=sheet_nm ,skiprows=3, nrows=9, usecols=lambda column: column != 'Unnamed: 0')
  mens_class = ((mens_class_base.drop(index=0)).reset_index(drop=True)).round(0)
  mens_class['Novice'] = list_rounder(list(mens_class.loc[:,'Novice'])) #Rounds the one row that did not automatically round for some reason
  mens_class['Category'] = mens_class['Category'].str.replace('kg', '', regex=False)
  womans_class_base = pd.read_excel(file_path, sheet_name=sheet_nm, skiprows=14, nrows=9, usecols=lambda column: column != 'Unnamed: 0')
  womans_class = ((womans_class_base.drop(index=0)).reset_index(drop=True)).round(0)
  womans_class['Novice'] = list_rounder(list(womans_class.loc[:,'Novice'])) #Rounds the one row that did not automatically round for some reason
  womans_class['Category'] = womans_class['Category'].str.replace('kg', '', regex=False)
  return mens_class, womans_class

def collsplit(df):
  """
  Splits a single column dataframe into a dataframe with multiple columns by space

  df: The dataframe we want to preform this action to
  """
  new_colnames = df.columns.str.split(expand=True)[0] #The zero is because it is a tuple
  split_df = df[df.columns[0]].str.split(expand=True)
  split_df.columns = new_colnames
  return split_df

def two_combine(df, col1, col2, newcolname):
  """
  Combines the two columns where both column enteries are seperated by a space in a new columns

  df: The dataframe
  col1: The column name of the leading column
  col2: The column name of the trailing column
  newcolname: The name of the new column
  """
  df[newcolname] = df[col1] + " " + df[col2]
  df.drop(columns=[col1, col2], inplace=True)
  return df

def upd_sheed_upload(sheet_path):
  """
  Uploads all of the tabs on the update sheet and makes the needed modifications

  sheet_path: The path to the update sheet
  """
  upd_sheet = pd.ExcelFile(sheet_path, engine='openpyxl')
  sheets = list(upd_sheet.sheet_names)
  sheet_dict = {}
  non_matched = {}
  for sheet in sheets:
    print(sheet)
    sheet_in = pd.read_excel(upd_sheet, sheet_name=sheet)
    #print(sheet_in.columns)
    if sheet_in.shape[1] == 1: #We assume it is a pdf form
      if list(sheet_in.columns)[0] == "Lot Last_Name First_Name Group Sex W.C. B.W. Team Born Snatch_1 Snatch_2 Snatch_3 Snatch_max Clean&Jerk_1 Clean&Jerk_2 Clean&Jerk_3 Clean&Jerk_max Total Sinclair Rank":
        df_update = collsplit(sheet_in)
        df_update = two_combine(df_update, "Sex", "W.C.", "Cat")
        #df_update.drop(columns=["Group"], inplace=True)
        sheet_dict[sheet] = df_update
      elif list(sheet_in.columns)[0] == "Lot Last_Name First_Name Cat. W.C. B.W. Team Born Snatch_1 Snatch_2 Snatch_3 Snatch_max R1 Clean&Jerk_1 Clean&Jerk_2 Clean&Jerk_3 Clean&Jerk_max R2 Total Rank":
        df_update = collsplit(sheet_in)
        df_update = two_combine(df_update, "Cat.", "W.C.", "Cat")
        sheet_dict[sheet] = df_update
      else:
        print("Warning: ", sheet, " Does not match any known format for single column initial")
        non_matched[sheet] = sheet_in
    elif sheet_in.shape[1] in [16, 18]: #Assuming possible OWLCSM forms
      upd_to_upd = upd_sheet_mod1(sheet_in)
      sheet_dict[sheet] = upd_to_upd
    elif sheet_in.shape[1] == 17: #Assuming a non OWLCSM
      #upd_to_upd = upd_sheet_mod1(sheet_in)
      sheet_dict[sheet] = sheet_in
    #elif sheet_in.shape[1] == 20: #This is one of the clean sheets we initially got
    #  sheet_dict[sheet] = sheet_in
    else: #We add it to a different dataframe and print a warning
      print("Warning: ", sheet, " Does not match any known format")
      non_matched[sheet] = sheet_in
  return sheet_dict, non_matched

def unrec_frame(non_matched_frame):
  """
  Causes the code to stop if there a dataframes in which we do not reccognize the form

  non_matched_frame: The dictionary of non matched dataframes ie the ones we do not recognize
  """
  assert len(non_matched_frame) == 0

def match_closest(value, candidates, threshold=85):
  """
  Finds the closest string in a set of canidates

  value: The string we want to test
  Canidates: The list of strings we want to compare to
  Threshold: How simmilar strings can be and still return what is needed
  """
  if value in candidates:
    return (value, 100, np.nan)
  result = process.extractOne(value, candidates, scorer=fuzz.ratio)
  if result and result[1] >= threshold:
      return (value, result[1], result[0]) # matched value
  return "None"  # or return value if you want to preserve original

def simp_name_replacer(df):
  """
  Replaces the names of the columns that are stagnent

  df: The dataframe we know we want to do the replaceing for
  """
  #Set the simple names that we want to replace
  simp_repnames = ["Last_Name", "First_Name", "Cat", "B.W.", "Club"]
  #Setting weight class varients so we can identify What we need to replace here
  weightclass_variants = ["weightclass", "weight class", "weight_class", "wtclass", "wt class", "wclass", "w class", "w_class", "wgtclass", "wgt class", "wgt_class",
                          "wc", "weight-category", "weight category", "weight-cat", "weight cat", "wt.", "division", "class", "category", "cat.", "Cat", "Cat."]
  #Setting the body weight varients
  bodyweight_variants = ["bodyweight", "body weight", "body_weight", "bdyweight", "bweight", "bw", "wt", "weight", "weigh", "athlete weight", "competition weight", "weigh-in", "weighin",
    "wgt", "comp weight", "weightkg", "kg", "mass", "B.W.", "b.w."]
  #Setting the first name and last name varients
  first_name_varients = ["first name", "firstname", "first_name", "fname", "f name", "given name", "given_name", "first", "forename", "First_Name", "First Name", "First_name"]
  last_name_varients = ["last name", "lastname", "last_name", "lname", "l name", "surname", "family name", "family_name", "last", "Last_Name", "Last Name", "Last_name"]
  #Setting all of the team name varients
  team_varients = ["team", "Team", "teamname", "team_name", "Team_name", "Team", "Name" "Club", "club", "Team"]
  "Building the dictionary of names"
  var_list = [last_name_varients, first_name_varients, weightclass_variants, bodyweight_variants, team_varients]
  var_dict = {}
  for i in range(len(simp_repnames)):
    var_dict[simp_repnames[i]] = var_list[i]
  #Itterate over each of the column names and replace as we see if
  df_names = list(df.columns)
  for simp_name in simp_repnames:
    high_var = 0
    replace_name = ""
    canidates = var_dict[simp_name]
    for cname in df.columns:
      clost_tup = match_closest(cname, canidates, threshold=85)
      if clost_tup != "None":
        df_cname, per_clo, cl_in_list = clost_tup
        if per_clo > high_var:
          high_var = per_clo
          replace_name = df_cname
    if replace_name != "":
      df = df.rename(columns={replace_name: simp_name})
  return df

def column_samer(raw_upd_dict):
  """
  Makes sure that all of the column names of each dataframe in the dictionary is the same

  raw_upd_dict: The dictionary of dataframes we want to comvert the colnames to
  cols_we_want: The column names that we want both dataframes to have. For now we assume it is the input from the OWLMS system
  """
  #Set the columns we want to replace
  unif_cols = ["Lot", "Last_Name", "First_Name", "M/F", "Cat", "B.W.", "Team", "Born", "Snatch_1", "Snatch_2", "Snatch_3", "Snatch_max",
               "Clean&Jerk_1", "Clean&Jerk_2", "Clean&Jerk_3", "Clean&Jerk_max", "Total", "Sinclair", "Cat. Rank", "Q-masters"]
  #Do the replaceing that we want
  dfs_to_add = list(raw_upd_dict.keys())
  new_clean_dict = {}
  for key in dfs_to_add:
    df = raw_upd_dict[key]
    df_cols = list(df.columns)
    if df_cols != unif_cols:
        df = simp_name_replacer(df) #Start with some brunt force replacement
        if "Q-masters" not in df.columns: #Adding what we use for rank if it is not in the frame that we need
          df["Q-masters"] = np.nan
        if "Sinclair" not in df.columns:
          df["Sinclair" ] = np.nan
        new_clean_dict[key] = df
    else: #If it is the same we are good
      new_clean_dict[key] = df
  return new_clean_dict


#Functions for adding needed information to the update dictionary

def comp_age_adder(upd_sheet, comp_date, bd_col, newcol_name, newcol_name_2):
  """
  Adds the age that the athlete was at the competition to the sheet
  This function also adds the year of birth of the athlete to the sheet

  upd_sheet: The sheet we are updating and adding the athletes age to
  comp_date: The date string of the competition
  bd_col: The column that specifies the athletes birthdate
  newcol_name: The name of the column we want to add to the sheet for the athletes age
  newcol_name_2: The name of the column we want to add to the sheet for the athletes year of birth
  """
  comp_dt = datetime.strptime(comp_date, "%Y-%m-%d")
  comp_year = int(comp_dt.year)
  ages = []
  yob = []
  upd_rownum = upd_sheet.shape[0]
  upd_sheet = upd_sheet.reset_index(drop=True)
  for i in range(upd_rownum):
    birth_date = upd_sheet.loc[i, bd_col]
    #print(birth_date)
    try: #See if we have a year only
      birth_year = int(birth_date)
      ages.append(comp_year - birth_year)
      yob.append(birth_year)
      continue
    except: #See if we can convert to a timestamp
      birth_date = pd.Timestamp(birth_date)
    #if isinstance(birth_date, pd.Timestamp):
      birth_year = birth_date.year
      ages.append(comp_year - birth_year)
      yob.append(birth_year)
      continue
  upd_sheet[newcol_name] = ages
  upd_sheet[newcol_name_2] = yob
  upd_sheet = upd_sheet.reset_index(drop=True)
  return upd_sheet

def catagory_adder(upd_sheet, age_col, new_colname):
  """
  Adds the athletes catagory to the sheet

  upd_sheet: The sheet we are updating and adding the athletes age to
  age_col: The column that specifies the athletes age
  new_colname: The name of the column we want to add to the sheet
  """
  catagories = []
  upd_rownum = upd_sheet.shape[0]
  for i in range(upd_rownum):
    age_in_q =  float(upd_sheet.loc[i, age_col]) #Makes sure it is a number if possible
    if age_in_q < 17:
      catagories.append('YTH')
    elif age_in_q < 21:
      catagories.append('JR')
    elif age_in_q < 30:
      catagories.append('SR')
    elif isinstance(age_in_q , (int, float)): #If it is a number greater then 30
      catagories.append('MST')
    else:
      catagories.append(np.nan)
  upd_sheet[new_colname] = catagories
  upd_sheet = upd_sheet.reset_index(drop=True)
  return upd_sheet

def WC_clean(df, wc_col, plus_back):
  """
  Cleans up the weightclass column to be in the form we need
  This applied to the HW weight catagories where the + and > can be all over the place
  df: The dataframe we want to preform this action to
  wc_col: The column name of the weightclass column
  plus_back: Specifies if we want + at the back of each weightclass in the row
  """
  df[wc_col] = df[wc_col].str.replace('>', '+', regex=False) #Do the simplest change first
  if plus_back: #Moves the plus to the back of the dataframe if true
    df[wc_col] = df[wc_col].str.replace('+', '', regex=False) + \
                 df[wc_col].str.contains(r'\+').map(lambda x: '+' if x else '')
  return df

def mst_ageadd(df, gro_col, new_col, mst_agecats, ath_agecol):
  """
  Adds the masters age catagories to the sheet for the masters athletes

  df: The df we want to add this information to
  gru_col: The catagory that signifies what group the athlete is in
  new_col: The new colname we want to use
  mst_agecats: The list of masters age categories in numerical order. This is a string of integers separated by commas. The last element in this list should be int('inf')
  ath_agecol: The column that specifies the athletes age
  """
  df = df.reset_index(drop=True)
  upd_rownum = df.shape[0]
  mst_cat = []
  for i in range(upd_rownum):
    gro_in_q = str(df.loc[i, gro_col])
    age_in_1 = float(df.loc[i, ath_agecol])
    if gro_in_q == 'MST':
        for age in range(len(mst_agecats)):
            if age_in_1 < mst_agecats[age]:
                mst_cat.append(mst_agecats[age-1])
                break
    else:
        mst_cat.append(np.nan)
  df[new_col] = mst_cat
  df = df.reset_index(drop=True)
  return df

def col_seperater_space(df, col1, col2, to_split_col):
  """
  Splits one column in a dataframe to two seperate columns by a space
  Note we only split after the first occurence

  df: The dataframe we want to preform this action to
  col1: The column name of the leading column
  col2: The column name of the trailing column
  to_split_col: The column name we want to split
  """

  df[[col1, col2]] = df[to_split_col].str.split(r"\s+", n=1, expand=True)
  df = df.reset_index(drop=True)
  return df

def MFW_seperator(df, col1, col2, to_split_col):
  """
  Splits after M, F, or W in a column to two seperate columns
  Note we only split after the first occurence

  df: The dataframe we want to preform this action to
  col1: The column name of the leading column
  col2: The column name of the trailing column
  to_split_col: The column name we want to split
  """
  #df[[col1, col2]] = df[to_split_col].str.split(r"(?<=[MFW])\s+", n=1, expand=True)
  #df = df.reset_index(drop=True)
  df = df.reset_index(drop = True) #Doing manually bc it is not working the way I want
  colel1 = []
  colel2 = []
  for i in range(df.shape[0]):
    look_el = str(df.loc[i, to_split_col][0])
    if look_el == 'M':
      colel1.append('M')
      colel2.append(look_el.replace('M', ''))
    elif look_el == 'F':
      colel1.append('F')
      colel2.append(look_el.replace('F', ''))
    elif look_el == 'W':
      colel1.append('W')
      colel2.append(look_el.replace('W', ''))
    #else: #This is cancled out so we see if there are any mistakes or weird cols as there will be a missmatch in column size
    #  colel1.append(np.nan)
    #  colel2.append(np.nan)
  df[col1] = colel1
  df[col2] = colel2
  return df

def clean_cat_col(df, gro_col, sex_col, wc_col, mst_age, new_colname):
  """
  Adds the proper catagory column name to the dataframes

  df: The dataframe we want to add the catagory to
  gro_col: The catagory that signifies what group the athlete is in
  sex_col: The column that specifies the athletes sex
  wc_col: The column that specifies the athletes weight class
  mst_age: The age catagory
  new_colname: The name of the column we want to add to the sheet
  """
  df = df.reset_index(drop=True)
  upd_rownum = df.shape[0]
  cats = []
  for i in range(upd_rownum):
    gro_in_q = str(df.loc[i, gro_col])
    sex_in_q = str(df.loc[i, sex_col])
    wc_in_q = str(df.loc[i, wc_col])
    ms_in_q = df.loc[i, mst_age]
    if gro_in_q == 'Mst':
      str_append = str(sex_in_q + str(int(ms_in_q)) + ' ' + wc_in_q)
      cats.append(str_append)
    else:
      str_append = str(sex_in_q + ' ' + wc_in_q)
      cats.append(str_append)
  df[new_colname] = cats
  df = df.reset_index(drop=True)
  return df

def scale_factor_locator(scal_fact_df, age, sex):
    """
    Locates the masters scale factor we want to scale by

    scal_fact_df: The scale factor dataframe as defined above
    age: The intiger age we want to get the scale factor for
    sex: The sex of the athlete we want to get the scale factor for as M or F
    """
    scal_fact_df = scal_fact_df[scal_fact_df["Age"] == int(age)]
    scal_fact_df = scal_fact_df.reset_index(drop=True)
    if sex == "M":
        return scal_fact_df.at[0, "AgeFactor_Men"]
    elif sex == "F":
        return scal_fact_df.at[0, "AgeFactor_Women"]
    else:
        print("sex needs to be M or F, here comes an error")

def sinclair_Q_adder(upd_sheet, sinclair_table, Q_agef_sheet, Q_col, Q_mst_col, WC_sinc_col, Bw_sinc_col, age_col, tot_col, sex_col, bw_col, wc_col): #masters_men_table, masters_woman_table master_mens_table: The coefficent sheet for the mens weightclasses master_womans_table: The coeffcent sheet for the womans weightclasses,
  """
  Adds the sinclair and Q points columns score to the update sheet

  upd_sheet: The dataframe we want to add the sinclair and Q points to for masters
  Q_col: The column name that we want the Q points to be in
  Q_mst_col: The column name that we want the Q points for masters to be in
  WC_sinc_col: The column name that we want the sinclair to be in
  Bw_sinc_col: The column name that we want the body weight to be in
  age_col: The column name that that specifies athletes age
  tot_col: The column name that specifies athletes total
  sex_col: The column name that specifies the athletes sex
  bw_col: The column name that specifies the athletes body weight
  wc_col: The column name that specifies the athletes weight class
  """
  BW_sinclair = []
  WC_sinclair = []
  #Mst_sinclair = [] #Removed as the masters federations no longer use this
  Q_points = [] #We are only going by weightclass for now as our assocuation only uses Sinclair
  Q_masters = []
  #Will need to add BW Q-masters here into this dataframe
  upd_sheet = upd_sheet.reset_index(drop=True)
  upd_rownum = upd_sheet.shape[0]
  #print("Row number:", upd_rownum)
  #Setting up which rows to skip ie what strings are missing what we need in the correct form
  skip_row = []
  for j in range(upd_rownum):
    try:
      ath_age = float(upd_sheet.loc[j, age_col])
      ath_total = float(upd_sheet.loc[j, tot_col])
      ath_sex = str(upd_sheet.loc[j, sex_col])
      ath_BW = float(upd_sheet.loc[j, bw_col])
      ath_wtcl = str(upd_sheet.loc[j, wc_col])
    except:
      skip_row.append(j)
  #print(skip_row)
  #Update the values
  for i in range(upd_rownum):
    if i in skip_row: #Append nan for what will not complete
      #print(i)
      BW_sinclair.append(np.nan)
      WC_sinclair.append(np.nan)
      #Mst_sinclair.append('NaN')
      Q_points.append(np.nan)
      Q_masters.append(np.nan)
    else: #If we can, run the normal loop
      ath_age = float(upd_sheet.loc[i, age_col])
      ath_total = float(upd_sheet.loc[i, tot_col])
      ath_sex = str(upd_sheet.loc[i, sex_col])
      #print(ath_sex)
      ath_BW = float(upd_sheet.loc[i, bw_col])
      ath_wtcl = str(upd_sheet.loc[i, wc_col])
      #print(ath_wtcl)
      if '+' in ath_wtcl: #Set up so we can use appropriate sinclair for heavyweight
        sin = 1
      else:
        sin = 0
        ath_wtcl = float(ath_wtcl)
      if ath_sex == 'M':
        BW_sinclair.append(sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Men']), float(sinclair_table.loc[1, 'Men'])))
        Q_points.append(q_points_male(ath_total, ath_BW))
        #print(sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Men']), float(sinclair_table.loc[1, 'Men'])))
        #print(q_points_male(ath_total, ath_BW))
        #Append weight class sinclair
        if sin == 0:
          WC_sinclair.append(sinclair_total(ath_total, float(ath_wtcl), float(sinclair_table.loc[0, 'Men']), float(sinclair_table.loc[1, 'Men'])))
        elif sin == 1:
          WC_sinclair.append(ath_total)
        else:
          WC_sinclair.append(np.nan)
        #Append masters Q points if nessisary
        if ath_age >= 30:
          #5 #Place holder number
          scale_factM = scale_factor_locator(Q_agef_sheet, ath_age, 'M')
          qM_masters_val = q_points_male(ath_total, ath_BW) * scale_factM
          Q_masters.append(qM_masters_val)
          #age_const_val_m = age_constant_selector(masters_men_table, 'Age_low', 'Age_high', 'Age_coefficient', ath_age)
          #Mst_sinclair.append(masters_sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Men']), float(sinclair_table.loc[1, 'Men']), age_const_val_m, ath_age))
        else:
          #5
          Q_masters.append(np.nan)
          #Mst_sinclair.append('NaN')
      elif ath_sex == 'W':
        BW_sinclair.append(sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Woman']), float(sinclair_table.loc[1, 'Woman'])))
        Q_points.append(q_points_female(ath_total, ath_BW))
        #print(sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Men']), float(sinclair_table.loc[1, 'Men'])))
        #print(q_points_female(ath_total, ath_BW))
        if sin == 0:
          WC_sinclair.append(sinclair_total(ath_total, ath_wtcl, float(sinclair_table.loc[0, 'Woman']), float(sinclair_table.loc[1, 'Woman'])))
        elif sin == 1:
          WC_sinclair.append(ath_total)
        else:
          WC_sinclair.append(np.nan)
        if ath_age >= 30:
          #5
          scale_factF = scale_factor_locator(Q_agef_sheet, ath_age, 'F')
          qF_masters_val = q_points_female(ath_total, ath_BW) * scale_factF
          Q_masters.append(qF_masters_val)
          #age_const_val_w = age_constant_selector(masters_woman_table, 'Age_low', 'Age_high', 'Age_coefficient', ath_age)
          #Mst_sinclair.append(masters_sinclair_total(ath_total, ath_BW, float(sinclair_table.loc[0, 'Woman']), float(sinclair_table.loc[1, 'Woman']), age_const_val_w, ath_age))
        else:
          #5
          Q_masters.append(np.nan)
        # Mst_sinclair.append('NaN')
      else:
        BW_sinclair.append(np.nan)
        WC_sinclair.append(np.nan)
        #print(np.nan)
        #print(np.nan)
        #Mst_sinclair.append('NaN')
        Q_points.append(np.nan)
        Q_masters.append(np.nan)
  upd_sheet[Bw_sinc_col] = BW_sinclair
  upd_sheet[WC_sinc_col] = WC_sinclair
  #upd_sheet['Masters_Sinclair'] = Mst_sinclair
  upd_sheet[Q_col] = Q_points
  upd_sheet[Q_mst_col] = Q_masters
  upd_sheet = upd_sheet.reset_index(drop=True)
  skip_row = [] #Reset just in case
  return upd_sheet

def class_adder(upd_sheet, mens_class_sheet, womans_class_sheet, age_col, tot_col, sex_col, bw_col, wc_col, group_col, class_col):
  """
  Adds the athletes class to the sheet

  mens_class_sheet: The mens classification sheet
  womans_class_sheet: The womens classification sheet
  upd_sheet: The sheet we are updating and adding the athletes age to
  age_col: The column name that that specifies athletes age
  tot_col: The column name that specifies athletes total
  sex_col: The column name that specifies the athletes sex
  bw_col: The column name that specifies the athletes body weight
  wc_col: The column name that specifies the athletes weight class
  group_col: The column name we want to add to the sheet for the age group
  class_col: The column name we want to add to the sheet for classification
  """
  #Set the catagories and classes
  jr_cats = ['Jr. Nats', 'Next Gen. Elite'] #Note that we may need to remove the space here
  sr_comp_groups = ['SR', 'MST']
  cols_to_remove = ['Category', 'Marker']
  upd_sheet = upd_sheet.reset_index(drop=True)
  upd_rownum = upd_sheet.shape[0]
  #Figure out the columns we need to skip
  skip_row = []
  for j in range(upd_rownum):
    try:
      ath_age = float(upd_sheet.loc[j, age_col])
      ath_total = float(upd_sheet.loc[j, tot_col])
      ath_sex = str(upd_sheet.loc[j, sex_col])
      ath_BW = float(upd_sheet.loc[j, bw_col])
      ath_wtcl = str(upd_sheet.loc[j, wc_col])
    except:
      skip_row.append(j)
  #Appending the classes
  classes = []
  for i in range(upd_rownum):
    if i in skip_row: #Append nan for what will not complete
      #print(i)
      classes.append('NaN')
    else: #If we can, run the normal loop
      ath_age = float(upd_sheet.loc[i, age_col])
      ath_total = float(upd_sheet.loc[i, tot_col])
      ath_sex = upd_sheet.loc[i, sex_col]
      ath_BW = float(upd_sheet.loc[i, bw_col])
      ath_wtcl = upd_sheet.loc[i, wc_col]
      ath_comp_group = upd_sheet.loc[i, group_col]
      if ath_sex == 'M':
          Msheet_to_det = (mens_class_sheet[mens_class_sheet['Category'] == ath_wtcl])
          Msheet_to_det = Msheet_to_det.drop(['Category'], axis=1) #Dont need this any more after weightclass selected
          pref_class = list(Msheet_to_det.columns)
          if ath_comp_group in sr_comp_groups:
            Msheet_to_det =  Msheet_to_det.drop(jr_cats, axis=1)
            pref_class = list(Msheet_to_det.columns)
          Msheet_to_det = (Msheet_to_det[pref_class]).reset_index(drop=True)
          if Msheet_to_det.shape[0] == 0:
            classes.append('NaN')
          else:
            for j in range(len(pref_class)-1):
              classif_b = list((Msheet_to_det.columns))[j]
              classif_t = list((Msheet_to_det.columns))[j+1]
              val_bot = float(Msheet_to_det.loc[0, classif_b])
              val_top = float(Msheet_to_det.loc[0, classif_t])
              if (classif_b == 'Novice') & (val_bot > ath_total):
                classes.append(classif_b)
                break
              elif (classif_t == 'Elite') & (val_top < ath_total):
                classes.append(classif_t)
                break
              elif val_bot <= ath_total < val_top:
                classes.append(classif_b)
                break
      elif ath_sex == 'W':
        Wsheet_to_det = (womans_class_sheet[womans_class_sheet['Category'] == ath_wtcl])
        Wsheet_to_det = Wsheet_to_det.drop(['Category'], axis=1)
        pref_class = list(Wsheet_to_det.columns)
        if ath_comp_group in sr_comp_groups:
          Wsheet_to_det =  Wsheet_to_det.drop(jr_cats, axis=1)
          pref_class = list(Wsheet_to_det.columns)
        Wsheet_to_det = (Wsheet_to_det[pref_class]).reset_index(drop=True)
        if Wsheet_to_det.shape[0] == 0:
          classes.append('NaN')
        else:
          for j in range(len(pref_class)-1):
            classif_b = list((Wsheet_to_det.columns))[j]
            classif_t = list((Wsheet_to_det.columns))[j+1]
            val_bot = float(Wsheet_to_det.loc[0, classif_b])
            val_top = float(Wsheet_to_det.loc[0, classif_t])
            if (classif_b == 'Novice') & (val_bot > ath_total):
              classes.append(classif_b)
              break
            elif (classif_t == 'Elite') & (val_top < ath_total):
              classes.append(classif_t)
              break
            elif val_bot <= ath_total < val_top:
              classes.append(classif_b)
              break
  upd_sheet[class_col] = classes
  upd_sheet = upd_sheet.reset_index(drop=True)
  return upd_sheet

def srjryth_rem(df, colname):
  """
  Removes the group entery from the catagory column in the cleaining

  df: The dataframe we want to remove the group from
  colname: The name of the column we want to remove the group from
  """
  new_col = []
  df = df.reset_index(drop=True)
  upd_rownum = df.shape[0]
  for i in range(upd_rownum):
    cat_in_q = str(df.loc[i, colname])
    #print(cat_in_q)
    if "SR" in cat_in_q:
      new_col.append(cat_in_q.replace("SR ", "")) #The extra space gets rid of any error when the next function happens
    elif "MST" in cat_in_q:
      new_col.append(cat_in_q.replace("MST ", ""))
    elif "JR" in cat_in_q:
      new_col.append(cat_in_q.replace("JR ", ""))
    elif "YTH" in cat_in_q:
      new_col.append(cat_in_q.replace("YTH ", ""))
    else:
      new_col.append(cat_in_q)
  df[colname] = new_col
  return df

def sex_cleaner(df, sex_col):
  """
  Quality controls the sex column to prevent errors down the line

  df: The dataframe we want to clean
  sex_col: The name of the column we want to clean
  """
  df = df.reset_index(drop=True)
  upd_rownum = df.shape[0]
  new_col = []
  for i in range(upd_rownum):
    sex_in_q = str(df.loc[i, sex_col])
    if 'M' in sex_in_q:
      new_col.append('M')
    elif 'F' in sex_in_q:
      new_col.append('F')
    elif 'W' in sex_in_q:
      new_col.append('W')
    else:
      new_col.append(np.nan)
  df[sex_col] = new_col
  return df

def upd_dict_infoadd(upd_dict, sinc_const_sheet, m_class_sheet, f_class_sheet):
  """
  Adds all of the unformation to the update dictionarys that we need to update both the records and rankings sheets

  upd_dict: The dictionary of update dataframes
  sinc_const_sheet: The sinclair constant sheet
  m_class_sheet: The mens classification sheet
  f_class_sheet: The womens classification sheet
  """
  new_dict = {}
  upd_dict_keys = list(upd_dict.keys())
  for key in upd_dict_keys:
    print(key)
    #Get everything ready for some spicey manipulations
    date, comp = key.split("_")
    df = upd_dict[key].reset_index(drop=True)
    #Add the current age of the athlete
    compage_col = 'Comp_Age'
    year_of_birth_col = 'YOB'
    df = comp_age_adder(df, date, 'Born',  compage_col, year_of_birth_col)
    #Add the athlete catagory if it is not already in the dataframe
    cat_add_colname = 'Group'
    if cat_add_colname not in list(df.columns):
      df = catagory_adder(df, compage_col, cat_add_colname)
    #Add the masters age catagory for all masters lifters
    mast_agecol = "Mst_Age"
    mst_agewinds = [30, 35, 40, 45, 45, 50, 55, 60, 65, 70, 75, 80, 85, float('inf')]
    df = mst_ageadd(df, cat_add_colname, mast_agecol, mst_agewinds, compage_col)
    #Generate seperate sex and weightclass col we assume that
    sex_col = 'Sex'
    wc_col = 'W.C.'
    if 'Cat' in list(df.columns):
      df = srjryth_rem(df, 'Cat')
    try:
      df = col_seperater_space(df, sex_col, wc_col, 'Cat')
    except:
      df[sex_col] = df['Cat']
      df = sex_cleaner(df, sex_col) #Literally bc it already exist for the masters so we will just clean it up
      #df = MFW_seperator(df, sex_col, wc_col, 'Cat') #Assume the error is caused by no space in the cat col so we are ging to do manually
    df[wc_col] = df[wc_col].astype(str) #This is because we will get NaNs in the next line of code that we do not want if we do not do this
    df = WC_clean(df, wc_col, True) #Replace for the new coluumn
    df = WC_clean(df, 'Cat', True) #Replace for the old
    df = sex_cleaner(df, sex_col) #Does repleat for masters which is ok here
    #Generates the proper column
    prop_cantname = "Prop_cat"
    df = clean_cat_col(df, cat_add_colname, sex_col, wc_col, mast_agecol, prop_cantname)
    #print(df)
    #Adding the sinclare score if it does not exist
    #print(df["Total"]) #print(df["Sex"]) #print(df["W.C."]) #print(df["B.W."]) #print(df["Comp_age"])
    bw_col = "B.W."
    tot_col = "Total"
    df = sinclair_Q_adder(df, sinc_const_sheet, Q_agef_sheet, "BW_Q-points", "BW_Q-masters", "WC_Sinclair", "BW_Sinclair",  compage_col, tot_col, sex_col, bw_col, wc_col)
    #print(df["BW_Q-points"]) #print(df["BW_Q-masters"] #print(df["WC_Sinclair"]) #print(df["BW_Sinclair"])
    #Add the athletes classes
    class_col = "Qualification_Standard"
    group_col = "Group"
    df = class_adder(df, m_class_sheet, f_class_sheet, compage_col, tot_col, sex_col, bw_col, wc_col, group_col, class_col)
    #Adding the comp date and name to the actual dataframe
    df["Comp_Date"] = date
    df["Comp_Name"] = comp
    #One very simple update
    last_name_col = "Last_Name"
    df[last_name_col] = df[last_name_col].str.upper()
    #Dropping all 98 weightclass values
    df = df[df[wc_col] != '98']
    df = df[df[wc_col] != 98]
    #Adding the sheet to the dictionary
    new_dict[key] = df
  #Returning the updated dictionary with the information
  return new_dict


#Functions to update the ranking sheets

def rank_adder(df, rank_cname):
  """
  Adds the rank column to the dataframe

  df: The dataframe we want to add the rank column to
  rank_cname: The name of the column we want to add for rank
  """
  if rank_cname in list(df.columns): #Get rid of the old column
    df = df.drop(rank_cname, axis=1)
  df = df.reset_index(drop=True)
  rank_col = []
  for i in range(1, df.shape[0] + 1):
    rank_col.append(i)
  df.insert(0, "Rank", rank_col) #Makes sure that the rank is at the front of the dataframe
  return df

def duplicate_remover(df, firstname_colname, lastname_colname, wtclass_colname, rank_col):
  """
  Builds a dataframe that has highest entery for a person by weightclass
  Basically, this function removes all of the unessisary duplicates in the rankings sheet
  Ensure the relevant columns have a consistent format (strings for names, weight class) or this function will not work as good as you need

  df: The dataframe we are doing this operation on
  firstname_colname: The name of the first name column
  lastname_colname: The name of the last name column
  wtclass_colname: The name of the weight class column
  rank_col: The name of the column that we have used for rankings
  """
  df[firstname_colname] = (df[firstname_colname].astype(str)).str.strip()
  df[lastname_colname] = (df[lastname_colname].astype(str)).str.strip()
  df[wtclass_colname] = (df[wtclass_colname].astype(str)).str.strip()
  df[rank_col] = pd.to_numeric(df[rank_col], errors='coerce')  #Ensure rank is numeric
  #Sort the DataFrame by rank_col in descending order (higher ranks come first)
  df_sorted = df.sort_values(by=rank_col, ascending=False)
  #Drop duplicates based on the combination of firstname, lastname, and wtclass
  #Keeps the first occurrence (highest rank due to sorting)
  cleaned_df = df_sorted.drop_duplicates(subset=[firstname_colname, lastname_colname, wtclass_colname], keep='first').reset_index(drop=True)
  return cleaned_df

def rankings_updator(rank_to_upd, upd_dict, rank_col, sex_col, upd_keyname_dict, date_soon, date_end): #m_ranksheet_name, f_ranksheet_name
  """
  Updates the rankings sheets in the rank_to_upd dictionary by the rank col
  We add from all of the competitions in the update dictionary

  rank_to_update: The dictionary that contains the current rankings before the updates
  upd_dict: The dictionary that contains the competitions sheets to add
  rank_col: The column we want to do the ranking by
  update_keyname_dict: The dictionary that specifies what key goes with male, female, and other oprientation rankings in the current ranking dictionary.
                       This key name is usually the sheet name in the rankings dictionary
  date_soon: The earlies date we want to include in the sheet
  date_end: The latest date we want to include in the sheet
  """
  #Getting the needed information and sheets from the current rankings sheet
  curr_male_ranks = rank_to_upd[upd_keyname_dict['Male_Ranks']]
  curr_male_cnames = list(curr_male_ranks.columns)
  curr_female_ranks = rank_to_upd[upd_keyname_dict['Female_Ranks']]
  curr_female_cnames = list(curr_female_ranks.columns)
  #Setting up the update dictionary to be in the correct form to be incorperated into the current rankings
  upd_stacked = pd.concat(upd_dict.values(), ignore_index=True)
  #Filtering to only contain the dates we want
  #upd_stacked['Comp_Date'] = pd.to_datetime(upd_stacked['Comp_Date'])
  #upd_stacked['Comp_Date'] = upd_stacked['Comp_Date'].dt.date
  upd_stacked[rank_col] = upd_stacked[rank_col].astype(float)
  upd_stacked = upd_stacked[upd_stacked['Comp_Date'] >= date_soon]
  upd_stacked = upd_stacked[upd_stacked['Comp_Date'] <= date_end]
  #upd_stacked = upd_stacked.reset_index(drop=True) #Doing just in case
  upd_male = upd_stacked[upd_stacked[sex_col] == 'M'].reset_index(drop=True)
  unrm_male = upd_male[curr_male_cnames].dropna(subset=[rank_col])
  upd_female = upd_stacked[upd_stacked[sex_col] == 'W'].reset_index(drop=True)
  unrf_female = upd_female[curr_female_cnames].dropna(subset=[rank_col])
  #Organizing and cleaning for the new rankings
  #Male
  male_stacked = pd.concat([curr_male_ranks, unrm_male], axis=0, ignore_index=True)
  male_sorted = male_stacked.sort_values(by=rank_col, ascending=False)
  male_no_dups = duplicate_remover(male_sorted, "First_Name", "Last_Name", "W.C.", rank_col)
  male_ranked = rank_adder(male_no_dups, "Rank").reset_index(drop=True)
  male_ranked["Total"] = male_ranked["Total"].astype(int) #Some last minute cleaning
  male_ranked[rank_col] = male_ranked[rank_col].round(2)
  #Female
  female_stacked = pd.concat([curr_female_ranks, unrf_female], axis=0, ignore_index=True)
  female_sorted = female_stacked.sort_values(by=rank_col, ascending=False)
  female_no_dups = duplicate_remover(female_sorted, "First_Name", "Last_Name", "W.C.", rank_col)
  female_ranked = rank_adder(female_no_dups, "Rank").reset_index(drop=True)
  female_ranked["Total"] = female_ranked["Total"].astype(int) #Some last minute cleaning
  female_ranked[rank_col] = female_ranked[rank_col].round(2)
  #print(female_ranked)
  #Return the new updated dataframe
  rank_to_upd[upd_keyname_dict['Male_Ranks']] = male_ranked
  rank_to_upd[upd_keyname_dict['Female_Ranks']] = female_ranked
  return rank_to_upd

def upd_inx_dict(col_list):
  """
  Builds the dictionary that assignes the column names with the index so we can update the correct values

  col_list: The list of column names we want to update
  """
  upd_dict = {}
  alphabet = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
  for i in range(len(col_list)):
    upd_dict[col_list[i]] =  alphabet[i]
  return upd_dict

def rank_sheet_update(pathID, subsheet, replace_sheet, row_start, quota):
  """
  Updates the rankings sheet in the google sheet

  pathID: The file id of the .gsheet files or the full path of the .xlxs file
  subsheet: The subsheet we want to update and replace
  auth: Specifies if we need to remount to the drive or authenticate the user for the google sheet upload. "True" Means that we do
  replace_sheet: The sheet we want to update the corrent records with
  row_start: The row we want to start updating from in the update sheet
  quota: The amount of cells we can update per minute, stupid google

  This is a very usefull think for the back half of the function https://www.youtube.com/watch?v=cN7W2EPM-dw
  This is also very useful if you like reading https://docs.gspread.org/en/latest/index.html
  """
  #Set needed packages
  from google.colab import auth
  import google.auth
  #Authenticate and open the sheet
  auth.authenticate_user()
  creds, _ = google.auth.default()
  gc = gspread.authorize(creds)
  sheet_file = gc.open_by_key(pathID).worksheet(subsheet) #Open the very specific worksheet
  #Updating the first row with the update date and time (Note we get lucky and it keeps the origional format)
  col_date_name = "Updated as of: " + str(datetime.now().strftime("%Y-%m-%d"))
  sheet_file.update([[col_date_name]], "A1")
  #Updating the meat of the dataframe
  values_list = list(sheet_file.row_values(2))
  upd_dict = upd_inx_dict(values_list)
  count = 0
  print("Updating ", subsheet)
  for val in values_list:
    upd_els = list(replace_sheet[val]) #Note that we assume that the column names for the dataframe are the same as list we want to update
    col_ind = upd_dict[val]
    for i in range(len(upd_els)):
      update_indx = str(col_ind + str(row_start+i))
      sheet_file.update_acell(update_indx, upd_els[i])
      count += 1
      if count == quota:
        print("Taking a 60 second pause to stay under the quota")
        time.sleep(60)
        count = 0
  print(subsheet, " updated")


#Some extra functions that may be used

def list_string_split(the_list, split_el, num_seps):
  """
  Splits all strings in a list of strings by another string

  list: The list of strings we want to split
  split_el: The string we want to split the strings by
  num_seps: The number of times we want to split the string, Note that it goes from front to back on where it splits
  """
  new_list = []
  for i in range(len(the_list)):
    new_list.append(the_list[i].split(split_el, num_seps))
  return new_list

def list_unpacker(the_list, sub_el_index):
  """
  This function makes a new list of a specified element of a sublist in a list of list

  the_list: The list of lists we want to unpack
  sub_el_index: The index of the element we want to unpack
  """
  new_list = []
  for i in range(len(the_list)):
    new_list.append(the_list[i][sub_el_index])
  return new_list

def rankings_seperator(df, age_group_colname, age_groups):
  """
  Breaks up a rankings sheet into groups

  df: The dataframe we want to break up
  age_group_colnames: The column name that specifies the age groups
  age_groups: The age groups within the age group column names that we want to sperate the data frame into
  """
  df_split = df[df[age_group_colname].isin(age_groups)]
  df_split = df_split.reset_index(drop=True)
  df_split = df_split.drop('Rank', axis=1)
  df_split = rank_adder(df_split)
  df_split = df_split.drop(age_group_colname, axis=1)
  return df_split


#Code to update records sheets

def class_cat_dbuild(sheet_names, cat_clases):
  """
  Builds the dictionary that assignes the name of the sheet with the category

  sheet_names: The record sheet names as defined above
  class_cat: The class catagories as defined as follows. 'YTH_W':... , 'YTH_M':... , 'JR_W':... , 'JR_M':... , 'SR_W':... , 'SR_M':... , 'MST_M':... , 'MST_W':...
  """
  class_sheet_dict = {}
  for i in range(len(sheet_names)):
    class_sheet_dict[cat_clases[i]] = sheet_names[i]
  return class_sheet_dict

def list_string_split(the_list, split_el, num_seps):
  """
  Splits all strings in a list of strings by another string

  list: The list of strings we want to split
  split_el: The string we want to split the strings by
  num_seps: The number of times we want to split the string, Note that it goes from front to back on where it splits
  """
  new_list = []
  for i in range(len(the_list)):
    new_list.append(the_list[i].split(split_el, num_seps))
  return new_list

def list_unpacker(the_list, sub_el_index):
  """
  This function makes a new list of a specified element of a sublist in a list of list

  the_list: The list of lists we want to unpack
  sub_el_index: The index of the element we want to unpack
  """
  new_list = []
  for i in range(len(the_list)):
    new_list.append(the_list[i][sub_el_index])
  return new_list

def date_key_sort(the_keys, orderd_dates):
  """
  Sorts the list of dataframe keys for the updated competitions from soonest to latest

  the_keys: The keys we want to sort
  orderd_dates: The list of dates we want to order the keys by
  """
  new_keys = []
  for date in orderd_dates:
    for key in the_keys:
      if date in key:
        new_keys.append(key)
  return new_keys

def proper_null_convert(df, col_list):
  """
  Converts all of the columns that have null lifts into the proper form to compute the records

  df: The dataframe we want to do this form
  col_list: The list of columns we want to convert
  """
  pattern = r"\(\d+\)"
  for col in col_list:
    if df[col].astype(str).str.contains(pattern, na=False).any(): #Updates if it exist in the dataframe
      df.loc[:,col] = df[col].str.replace(r"\((\d+)\)", r"-\1", regex=True) #Note that we can add more forms to this function as we come across more
      df.loc[:,col] = pd.to_numeric(df[col], errors="coerce")
    else: #Note that we will likely have to update this function as we get more cases of no lifts that are not negaitives
      df.loc[:,col] = pd.to_numeric(df[col], errors="coerce")
  return df

def max_min_finder(df, col_list):
  """
  Finds the min and max lift in each comp to save us some itterations when we go through the sheet for records
  Note that this uses the absolute values of each one

  df: The dataframe we want to do this form
  col_list: The list of columns we want to finds the min and max lifts for
  """
  max_val = 0 #float('-inf')
  min_val = float('inf')
  for col in col_list:
    nums_to_test = list(df[col])
    for num in nums_to_test:
      if isinstance(num, (int, float)) == False:
        continue
      if abs(num) > max_val:
        max_val = abs(num)
      if abs(num) < min_val:
        min_val = abs(num)
  return min_val, max_val

def rec_filt_error(df):
  """
  Stops the code if the record sheet is not filterd for a single error

  df: The record datasheet we want to pull the error on
  """
  assert df.shape[0] == 1, "The record sheet is not filtering for a single record"

def brac_rec_rem(el):
  """
  Puts the record into the correct form to be compared with the athletes made attempt

  el: The element we want to convert to the correct form
  """
  el_type = type(el)
  if el_type == int:
    return el
  elif el_type == str:
    if "[" in el:
      el = el.replace("[", "")
    if "]" in el:
      el = el.replace("]", "")
      #Note that we will need to add more cases as they come
    return int(el)
  else:
    raise ValueError(f"Unsupported element type: {el_type}")

def update_sequence(rec_dic, clean_sheet, class_sheet_dic, rsheet_key, last_name, first_name, corg_val, wc_val, lift, club, the_try, comp_name, comp_date):
  """
  Updates the record sheet with the nessisary information

  clean_sheet: The sheet we want to update the records from
  rec_dic: The dictionary of the current records sheets
  class_sheet_dict: The dictionary that lets us know what records sheet goes with what category
  last_name: The athlete last name
  first_name: The athlete first name
  corg_val: The category value we need to put in to the right record sheet
  wc_val: The weight class value we need to put in to the right record sheet
  lift: The lift we are doing the records updates for
  club: The club we are doing the records updates for
  the_try: The attempt we are doing the records updates for
  comp_name: The name of the competition we are doing the records updates for
  comp_date: The date of the competition we are doing the records updates for
  """
  rsheet_in_q = rec_dic[class_sheet_dic[rsheet_key]]
  rsheet_in_q = rsheet_in_q.reset_index(drop = True)
  #print(corg_val, wc_val, lift)
  mst_upd_sheet = rsheet_in_q[(rsheet_in_q["Category"] == corg_val) & (rsheet_in_q["Weight Cat."] == wc_val) & (rsheet_in_q["Lift"] == lift)]
  rec_filt_error(mst_upd_sheet) #Making sure there are no problems here
  row_index = mst_upd_sheet.index.tolist()[0]
  rec_val = mst_upd_sheet.loc[row_index, "Record"]
  rec_val = brac_rec_rem(rec_val)
  if rec_val < the_try: #Append the new record and add to the historical reccords
    replacement_row = {'Category':corg_val, 'Weight Cat.':wc_val, 'Lift':lift, 'Last Name':last_name,
                       'First Name':first_name, 'Club':club, 'Record':int(the_try), 'Event':comp_name,
                       'Location':np.nan, 'Date':comp_date}
    rsheet_in_q.loc[row_index] = replacement_row
    rec_dic[class_sheet_dic[rsheet_key]] = rsheet_in_q #Replace the value in the sheet
    hist_frame = rec_dic[class_sheet_dic['HIST']]
    new_hist_frame = pd.concat([hist_frame, mst_upd_sheet], ignore_index=True).reset_index(drop = True) #Note that we will sort after
    rec_dic[class_sheet_dic['HIST']] = new_hist_frame
  return rec_dic

def reccord_sequence(rec_dic, clean_sheet, min_val, max_val, wt_clsses, class_sheet_dic, cols_to_itter, lift):
  """
  Updates the records from a given comp sheet

  rec_dict: The dictionary of the current records sheets
  clean_sheet: The sheet we want to update the records from
  min_val: The smallest attempt in the sheet
  max_val: The largest attempt in the sheet
  wt_clsses: The list of weight classes we want to update the records for
  class_sheet_dic: The dictictionary that lets us know what records sheet goes with what category
  cols_to_itter: The list of columns we want to itterate through to find the records, it has to be in order from sn1-sn3 then cj1-cj3
  lift: The lift we are doing the records updates for
  """
  for wt_cl in wt_clsses:
    #There is no reccord for this weightclass beccause of the change so we skip
    if wt_cl == '98' or wt_cl == int(98):
      continue
    curr_sheet = clean_sheet[clean_sheet['W.C.'] == wt_cl]
    curr_sheet = curr_sheet.sort_values(by='Lot', ascending=True) #Making sure we are in order again just in case, make sure this variable is numeric
    #print(curr_sheet)
    curr_sheet = curr_sheet.reset_index(drop=True) #Having the index reset will allow us to refer back to rows when we need
    #print(curr_sheet)
    #Starting with snatches
    for att in range(int(min_val), int(max_val+1)): #We use integers as there are not half value attemps
      next = False #Settin the varible to tell us we we must go to the next attemps
      for cols in cols_to_itter:
        ct_inv = list(curr_sheet[cols])
        for i in range(len(ct_inv)):
          the_try = ct_inv[i]
          #print(the_try)
          if the_try == att: #Note that we move past negitives which are failed attempts, or null enteries
            #print(the_try, att)
            #Set up for the next itteration
            next = True
            #Getting the needed values for the update
            lot_num = list(curr_sheet['Lot'])[i]
            last_name = list(curr_sheet['Last_Name'])[i]
            first_name = list(curr_sheet['First_Name'])[i]
            comp_date = list(curr_sheet['Comp_Date'])[i]
            comp_date =  pd.to_datetime(comp_date,  errors="raise").strftime("%Y-%m-%d") #Make sure it is in the right form
            comp_name = list(curr_sheet['Comp_Name'])[i]
            mst_age = list(curr_sheet['Mst_Age'])[i]
            sex = list(curr_sheet['Sex'])[i]
            group = list(curr_sheet['Group'])[i]
            yob = list(curr_sheet['YOB'])[i]
            prop_cat = list(curr_sheet['Prop_cat'])[i]
            club = list(curr_sheet['Club'])[i]
            #Checking to see if we need to update
            rsheet_key = group + "_" + sex
            rsheet_key = re.sub(r"\s+", "", str(rsheet_key)) #Removing just in case
            rsheet_in_q = rec_dic[class_sheet_dic[rsheet_key]]
            #print(rsheet_in_q)
            rsheet_in_q = rsheet_in_q.reset_index(drop = True) #Making sure we can cleanly refer back to what we need to each time
            wc_use = str(wt_cl) + "kg" #The weightclass in the sheet
            if '+' in wc_use: #Make sure the heavyweight classes are in the proper form
              wc_use = "+" + wc_use.replace("+", "", 1)
            #Handeling each update case
            if group == "MST":
              #For the masters sheet
              cat_val = sex + str(int(mst_age))
              rec_dic = update_sequence(rec_dic, clean_sheet, class_sheet_dic, rsheet_key, last_name, first_name, cat_val, wc_use, lift, club, the_try, comp_name, comp_date)
              #Need to update SR as well
              rsheet_key2 = "SR" + "_" + sex
              rec_dic = update_sequence(rec_dic, clean_sheet, class_sheet_dic, rsheet_key2, last_name, first_name, "SR", wc_use, lift, club, the_try, comp_name, comp_date)
              break
            elif group == "JR":
              #For the junior sheet
              rec_dic = update_sequence(rec_dic, clean_sheet, class_sheet_dic, rsheet_key, last_name, first_name, group, wc_use, lift, club, the_try, comp_name, comp_date)
              rsheet_key2 = "SR" + "_" + sex
              #Need to update for SR as well
              rec_dic = update_sequence(rec_dic, clean_sheet, class_sheet_dic, rsheet_key2, last_name, first_name, "SR", wc_use, lift, club, the_try, comp_name, comp_date)
              break
            elif group == "YTH":
              #For the youth
              rec_dic = update_sequence(rec_dic, clean_sheet, class_sheet_dic, rsheet_key, last_name, first_name, group, wc_use, lift, club, the_try, comp_name, comp_date)
              #Note that we will need to update for youth athletes breaking JR and SR records
              break
            elif group == "SR":
              #For the senior
              rec_dic = update_sequence(rec_dic, clean_sheet, class_sheet_dic, rsheet_key, last_name, first_name, group, wc_use, lift, club, the_try, comp_name, comp_date)
            break
          else: #Business as usuall if we do not hit an attempt
            next = False
        #Stopping all innter itterations if we have found a record for this attempt
          if next == True:
            break
        if next == True:
          break
  #Return the updated record dictionary
  return rec_dic

def rec_sheet_update(rec_dic, upd_dic, class_sheet_dic):
  """
  This function updates the record sheets

  rec_dic: The dictionary of the current records sheets
  upd_dic: The dictionary of the new competitions into the sheet
  class_sheet_dic: The dictictionary that lets us know what records sheet goes with what category
  quota: The maximum amount of updates we can make to a google sheet in one operaion
  """
  #Gets the needed information to order the updates of the sheets
  upd_keys = list(upd_dic.keys())
  upd_split = list_string_split(upd_keys, '_', 1)
  dates = list_unpacker(upd_split, 0)
  dates_sorted = sorted(dates, key=lambda d: datetime.strptime(d, "%Y-%m-%d"))
  orderd_keys = date_key_sort(upd_keys, dates_sorted)
  sn_cols_to_test = ['Snatch_1', 'Snatch_2', 'Snatch_3']
  cj_cols_to_test = ['Clean&Jerk_1', 'Clean&Jerk_2', 'Clean&Jerk_3']
  tot_cols_to_test = ['Total']
  cols_to_test = sn_cols_to_test + cj_cols_to_test + tot_cols_to_test
  #print(orderd_keys)
  #Updating the records
  for comp in orderd_keys:
    comp_results = upd_dic[comp]
    comp_results["Lot"] = comp_results["Lot"].astype(int) #Makes sure we are in the right form
    #Doing some basic mods before we start finding the records
    comp_resrec = comp_results[['Lot', 'Last_Name', 'First_Name', 'Group', 'Club','Snatch_1', 'Snatch_2', 'Snatch_3', 'Snatch_max', 'Clean&Jerk_1',
                                'Clean&Jerk_2', 'Clean&Jerk_3', 'Clean&Jerk_max', 'Total', 'YOB', 'Mst_Age', 'Sex', 'W.C.', 'Prop_cat',
                                'Comp_Date', 'Comp_Name']]
    cc_comp_res = proper_null_convert(comp_resrec, cols_to_test)
    #Mens updates
    #Getting the dataframe in order
    mens_res = cc_comp_res[cc_comp_res['Sex'] == 'M']
    msort_res = (mens_res.sort_values(by='Lot', ascending=True)).reset_index(drop=True)
    #print(msort_res)
    m_wcl = list(msort_res['W.C.'].unique())
    #Gettin the max and mins of each lift
    sn_mens_min, sn_mens_max = max_min_finder(msort_res, sn_cols_to_test)
    cj_mens_min, cj_mens_max = max_min_finder(msort_res, cj_cols_to_test)
    to_mens_min, to_mens_max = max_min_finder(msort_res, tot_cols_to_test)
    rec_dic = reccord_sequence(rec_dic, msort_res, sn_mens_min, sn_mens_max, m_wcl, class_sheet_dic, sn_cols_to_test, "Snatch")
    rec_dic = reccord_sequence(rec_dic, msort_res, cj_mens_min, cj_mens_max, m_wcl, class_sheet_dic, cj_cols_to_test, "Clean&Jerk")
    rec_dic = reccord_sequence(rec_dic, msort_res, to_mens_min, to_mens_max, m_wcl, class_sheet_dic, tot_cols_to_test, "Total")
    #Womans updates
    womens_res = cc_comp_res[cc_comp_res['Sex'] == 'W']
    wsort_res = (womens_res.sort_values(by='Lot', ascending=True)).reset_index(drop=True)
    w_wcl = list(wsort_res['W.C.'].unique())
    sn_womens_min, sn_womens_max = max_min_finder(wsort_res, sn_cols_to_test)
    cj_womens_min, cj_womens_max = max_min_finder(wsort_res, cj_cols_to_test)
    to_womens_min, to_womens_max = max_min_finder(wsort_res, tot_cols_to_test)
    rec_dic = reccord_sequence(rec_dic, wsort_res, sn_womens_min, sn_womens_max, w_wcl, class_sheet_dic, sn_cols_to_test, "Snatch")
    rec_dic = reccord_sequence(rec_dic, wsort_res, cj_womens_min, cj_womens_max, w_wcl, class_sheet_dic, cj_cols_to_test, "Clean&Jerk")
    rec_dic = reccord_sequence(rec_dic, wsort_res, to_womens_min, to_womens_max, w_wcl, class_sheet_dic, tot_cols_to_test, "Total")
  return rec_dic

def rec_sheet_updates(sheet_path, rec_sheet, match_dict):
  """
  Updates the record sheets in the google sheet

  sheet_path: The full path to the excell sheet in googled drive
  rec_sheet: The dictionary that contains the updated record sheets
  match_dict: The dictionary that lets us know what records sheet goes with what category
  """
  #Set needed packages internally
  from google.colab import auth
  import google.auth
  #Authenticate and open the sheet
  auth.authenticate_user()
  #Update
  for key in list(match_dict.keys()):
    sheet = match_dict[key]
    print("Updating: ", sheet)
    repl_sheet = rec_sheet[sheet]
    old_sheet = pd.read_excel(sheet_path, sheet_name=sheet)
    title_text = old_sheet.columns[0] #Get the header of the dataframe to be replaced
    with pd.ExcelWriter(sheet_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
      repl_sheet.to_excel(writer, index=False, startrow=1, sheet_name=sheet)
      #Get workbook and worksheet
      workbook = writer.book
      worksheet = writer.sheets[sheet]
      #Place the title in the first cell (row=1, col=1)
      worksheet.cell(row=1, column=1, value=title_text)
      #Merge across all dataframe columns and bold
      worksheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(repl_sheet.columns))
      cell = worksheet.cell(row=1, column=1)
      cell.font = Font(bold=True, size=16)
      cell.alignment = Alignment(horizontal="center", vertical="center")
      #Define a thin black border and then add the boarder to the cells
      thin_border = Border(
      left=Side(style="thin", color="000000"),
      right=Side(style="thin", color="000000"),
      top=Side(style="thin", color="000000"),
      bottom=Side(style="thin", color="000000"))
      for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column):
        for cell in row:
          if isinstance(cell, MergedCell):
            continue  # skip the "phantom" merged cells
          cell.border = thin_border
          ## (optional) wrap text so borders look neat with long strings
          #cell.alignment = Alignment(wrap_text=True, vertical="top")
      #Makes the cells wider
      for col_idx, col_cells in enumerate(worksheet.iter_cols(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column), start=1):
        max_len = 0
        for cell in col_cells:
          if isinstance(cell, MergedCell):
            continue  # skip placeholder cells from merged ranges
          if cell.value is not None:
            max_len = max(max_len, len(str(cell.value)))
        worksheet.column_dimensions[get_column_letter(col_idx)].width = max_len + 1
    print("Updated: ", sheet)



In [ ]:
#6) 

#Run all update operations and updates
#Note that you may need to change some of the functions and there inputs depending if you are using a google sheet or an excell file
#The function that is not a base pandas or other stock uploader "sheet_dict_creater". It is in cell #5

#Upload each file needed for the updates as a dictionary, and get into a form that we can run the updates on
print("Uploading Required Sheets")

#Rankings sheets
rank_sheets = ["2025-26 ALL FEMALE", "2025-26 ALL MALE"]
rank_dict_in = sheet_dict_creater(rank_sheet_ID, rank_sheets, "gsheet", True)
#print(rank_dict_in)
rank_to_upd = rank_sheet_mod1(rank_dict_in)
#print(rank_to_upd)

#Records sheets (Note that the whole name is not avalilible for each tab here)
#xls = pd.ExcelFile(rec_sheet_path) #To get the actual sheet names
#print(xls.sheet_names)
rec_sheets = ['Womens Sr (2025+ weight classes', 'Mens Sr (2025+ weight classes)', 'Womans Jr (2025+ weight classes', 'Mens Jr (2025+ weight classes)', 'Womens Yth (2025+ weight classe', 'Mens Yth (2025+ weight classes)', 'Womens Masters (2025+ weight cl', 'Mens Masters (2025+ weight clas','Previous Records 2025+'] #Names that are stored in the sheet
#rec_sheets = ['Womens Sr (2025+ weight classes)', 'Mens Sr (2025+ weight classes)', 'Womans Jr (2025+ weight classes)',  'Mens Jr (2025+ weight classes)', 'Womens Yth (2025+ weight classes)', 'Mens Yth (2025+ weight classes)', 'Womens Masters (2025+ weight classes)', 'Mens Masters (2025+ weight classes)','Previous Records 2025+'] #Full names
rec_dict_in = sheet_dict_creater(rec_sheet_path, rec_sheets, "xlsx", False)
#print(rec_dict_in)
rec_to_upd = rec_sheet_mod1(rec_dict_in)

#Update sheets
upd_sheets, non_matched = upd_sheed_upload(res_to_add_path)
#print(upd_sheets) #-Remember that there will be different ways to see missed attemps ((), -, ...)
#print(non_matched) - Note that we will need to reorganize the sheet to see who hit what when. This is done by the lot number
unrec_frame(non_matched) #Making sure that we have every frame reccognized
cc_upd_dict = column_samer(upd_sheets)
#print(cc_upd_dict)

#Classification sheet
curr_clsheetname = "Classification_2025-2026"
male_classheet, female_classheet = classification_setup(class_sheet_path, curr_clsheetname)
#print(male_classheet)
#print(female_classheet)

#Sinclair constant sheet
sinc_sheetname = "Sinclair_Constants"
sinc_sheet = pd.read_excel(sinc_sheet_path, sheet_name=sinc_sheetname)
#print(sinc_sheet)

#Q-Points masters age factors upload
Qf_sheetname = "Q_Masters_Age_Factors"
Q_agef_sheet = pd.read_excel(Q_agef_sheet, sheet_name=Qf_sheetname)
#print(Q_agef_sheet)

#Adding needed information to the update dictionary so we can update the records and rankings
print("Organizing uploaded sheets")
cc_upd_dict_2 = upd_dict_infoadd(cc_upd_dict, sinc_sheet, male_classheet, female_classheet)
#print(cc_upd_dict_2)

#Updaing the internal ranking dataframe
print("Updating rankings sheet")
rank_dict_keys = {'Male_Ranks':'2025-26 ALL MALE', 'Female_Ranks':'2025-26 ALL FEMALE', 'Other_ranks':np.nan} #Update the values of the dictionary as the sheet changes, dont changes the keys
rank_column = "WC_Sinclair" #Change this if you want to do the ranking by a different standard
rank_to_upd = rankings_updator(rank_to_upd, cc_upd_dict_2, rank_column, "Sex", rank_dict_keys, '2025-06-01', '2026-05-31')
#print(rank_to_upd)

#Updating the internal records sheets
print("Updating records sheet")
cat_clases = ['SR_W', 'SR_M', 'JR_W', 'JR_M', 'YTH_W', 'YTH_M', 'MST_W', 'MST_M', 'HIST']
class_sheet_dict = class_cat_dbuild(rec_sheets, cat_clases)
#print(class_sheet_dict)
new_rec_sheets = rec_sheet_update(rec_to_upd, cc_upd_dict_2, class_sheet_dict)
#print(new_rec_sheets)

#Functions to update the ranking sheet
#Note that this cell takes a few minutes to run due to the google API request quotas
if onl_upd_rank == True:
  #Update the male sheet
  print("Updating rankings sheets")
  rank_sheet_update(rank_sheet_ID, rank_dict_keys["Male_Ranks"], rank_to_upd[rank_dict_keys["Male_Ranks"]], 3, 59)
  #Update the female sheet
  rank_sheet_update(rank_sheet_ID, rank_dict_keys["Female_Ranks"], rank_to_upd[rank_dict_keys["Female_Ranks"]], 3, 59)

#Updating the record sheet
if onl_upd_recs == True:
  print("Updating records sheets")
  rec_sheet_updates(rec_sheet_path, new_rec_sheets, class_sheet_dict)

